# Classificador de notícies

Aquest notebook conté el procés de fer fine-tuning de CatalanULMFit entrenat amb el dump de Wikipedia.
- Primer entrenarem tot el LM amb la nostra data
- Després agafarem només l'encoder i crearem un objecte classifier on top per a realitzar la tasca desitjada.

## Instal·lant dependències

In [ ]:
import os
from fastai import *
from fastai.text import *

import numpy as np
import pandas as pd

from pathlib import Path
from functools import partial

## Load dataset

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('data.csv', delimiter='|', error_bad_lines=False)
data

,Type,Article
0,SPORTS,Una terrible calamarsada ha castigat aquest di...
1,SPORTS,La bona resposta de l'afició ha acabat donant ...
2,SPORTS,Frida Eldebrink i Giedre Labuckiene (foto) es ...
3,SPORTS,"Barça i Bayern de Munic, els dos únics campion..."
4,SPORTS,L'equip Racing Point de Fórmula 1 va anunciar ...
...,...,...
59,SALUT,Les comarques gironines han registrat 9.263 ca...
60,SALUT,Les proves clíniques de la vacuna contra la co...
61,SALUT,Un equip d'investigadors del Consell Superior ...
62,SALUT,Un estudi dirigit per la Charité - Universität...


In [ ]:
data.shape
data['Article'] = data['Article'].str.replace("[^a-zA-Z]", " ")
data = data.dropna()
data.rename(columns={"Article": "content"})
data['valid']=0
data['valid'][:int(data.shape[0]*0.15)]=1
data.to_csv(os.path.join('.', 'train_news.csv'), header=None, index=None)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


## Preprocessing: Tokenitzar i estandaritzarització

In [ ]:
tokenizer = Tokenizer(lang='ca', n_cpus=5)
data_lm = TextLMDataBunch.from_csv('.', tokenizer=tokenizer, bs=16, csv_name='train_news.csv')
data_lm.save('data_lm')

## Fine-tuning per al classificador

### Fine-tuning del Language Model amb la nostra data

Primer, abans d'entrenar el classificador (només fent servir l'encoder) fem *fine-tuning* amb la nostra data. Tot i que el model ha estat entrenat amb un dataset de domini-general molt divers, probablement la nostra data té una distribució diferent.

In [ ]:
weights_pretrained='model_60k-vocab-2'
itos_pretrained='itos'
pretrained_data=(weights_pretrained, itos_pretrained)

In [ ]:
!ls models

itos.pkl  model_60k-vocab-2.pth


In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=pretrained_data, drop_mult=0)
learn.freeze()

UnpicklingError: ignored

In [ ]:
learn.lr_find()
learn.recorder.plot(skip_start=0)

NameError: ignored

In [ ]:
# entrenem el learner objecte un epoch
learn.fit_one_cycle(1, 1e-2)

In [ ]:
learn.save('head_pretrained')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, 1e-3, moms=(0.8,0.7))

In [ ]:
learn.save('lm_fine_tuned')
learn.save_encoder('ft_enc')

### Fine-tuning de l'encoder

Ara toca entrenar el classificador utilitzant només l'encoder. 

In [ ]:
data_class = TextClassDataBunch.from_csv('.', vocab=data_lm.train_ds.vocab, bs=16, csv_name='new_train.csv', tokenizer=tokenizer)
learn = text_classifier_learner(data_class, AWD_LSTM, drop_mult=0.3)
learn.load_encoder('ft_enc')
learn.freeze

In [ ]:
learn.lr_find()
learn.recorder.plot(skip_start=0)

In [ ]:
learn.model

In [ ]:
learn.fit_one_cycle(2, 1e-2, moms(0.8,0.7))

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4), 1e-2), moms=(0.8,0.7))

In [ ]:
learn.freeze_to(-3)
learn.fit_once_cycle(1, slice(5e-3/(2.6**4), 5e-3), momds=(0.8,0.7))

In [ ]:
learn.unfreeze()
learn.fit_once_cycle(2, slice(1e-3/(2.6**4), 1e-3), momds=(0.8,0.7))

In [ ]:
learn.save('classificador_noticies')